<a href="https://colab.research.google.com/github/JeonJongDae/dinner_stock_AI/blob/main/rf05_pytorch06_class_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torch import nn
import matplotlib.pyplot as plt
import torchvision

In [ ]:
data_train = torchvision.datasets.FashionMNIST(root='.', train=True, download=True)
x_train = data_train.data / 255
y_train = data_train.targets

data_valid = torchvision.datasets.FashionMNIST(root='.', train=False, download=True)
x_valid = data_valid.data / 255
y_valid = data_valid.targets

100%|██████████| 26421880/26421880 [00:02<00:00, 11834349.91it/s]


Extracting ./FashionMNIST/raw/train-images-idx3-ubyte.gz to ./FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 200047.64it/s]


Extracting ./FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:01<00:00, 3732784.01it/s]


Extracting ./FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 8282423.09it/s]

Extracting ./FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./FashionMNIST/raw



In [ ]:
dataset_train = list(zip(x_train.unsqueeze(1), y_train))
dataset_valid = list(zip(x_valid.unsqueeze(1), y_valid))

dataloader_train = torch.utils.data.DataLoader(dataset_train, batch_size=32, shuffle=True, drop_last=True)
dataloader_valid = torch.utils.data.DataLoader(dataset_valid, batch_size=32)

In [ ]:
class ConvBlock(nn.Module):
    def __init__(self):
        super().__init__()
    
    def forward(self, input):
        return input * 2

conv = ConvBlock()
conv(torch.tensor([1, 2, 3]))
#=====================================
dummy = torch.randn(32, 10, 28, 28)
conv = nn.Conv2d(10, 20, kernel_size=3, padding=1, stride=1)
x = conv(dummy)
batch = nn.BatchNorm2d(20)
x = batch(x)
relu = nn.ReLU()
x = relu(x)

class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super().__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.stride = stride
        self.conv = nn.Conv2d(self.in_channels, self.out_channels, kernel_size=3, padding=1, stride=self.stride, bias=False)
        self.batch = nn.BatchNorm2d(self.out_channels)
    
    def forward(self, input):
        x = self.conv(input)
        x = self.batch(x)
        x = torch.relu(x)
        return x

conv = ConvBlock(10, 20)
conv.in_channels
z1 = conv(dummy)
z2 = conv(dummy)
torch.all(z1 == z2)

tensor(True)

In [ ]:
in_ch = 10
out_ch = 10
stride = 1

conv1 = nn.Conv2d(in_ch, out_ch, kernel_size=3, padding=1, stride=stride)
batch1 = nn.BatchNorm2d(out_ch)

conv2 = nn.Conv2d(out_ch, out_ch, kernel_size=3, padding=1)
batch2 = nn.BatchNorm2d(out_ch)

input = torch.randn(32, in_ch, 28, 28)
x = conv1(input) #
x = batch1(x)
x = torch.relu(x)
x = conv2(x)
x = batch2(x)
if in_ch == out_ch and stride == 1:
    x = x + input
x = torch.relu(x)
x.shape

debug


torch.Size([32, 10, 28, 28])

In [ ]:
class ResBlock(nn.Module):
    def __init__(self, in_ch, out_ch, stride=1):
        super().__init__()
        self.conv1 = nn.Conv2d(in_ch, out_ch, kernel_size=3, padding=1, stride=stride)
        self.batch1 = nn.BatchNorm2d(out_ch)

        self.conv2 = nn.Conv2d(out_ch, out_ch, kernel_size=3, padding=1)
        self.batch2 = nn.BatchNorm2d(out_ch)
    
    def forward(self, input):
        x = self.conv1(input) #
        x = self.batch1(x)
        x = torch.relu(x)
        x = self.conv2(x)
        x = self.batch2(x)
        if in_ch == out_ch and stride == 1:
            x = x + input
        x = torch.relu(x)
        return x

res = ResBlock(10, 20, 2)
res(input).shape

RuntimeError: ignored

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
for x, y in dataloader_train:
    break
x.shape, y.shape #(torch.Size([32, 1, 28, 28]), torch.Size([32]))

model = nn.Sequential(
    ConvBlock(1, 32), 
    ConvBlock(32, 64, stride=2),
    ConvBlock(64, 64),
    ConvBlock(64, 128, stride=2),
    ConvBlock(128, 128),
    # nn.Flatten(),
    # nn.Linear(6272, 128),
    # nn.ReLU(),
    # nn.Linear(128, 10),
    nn.AvgPool2d(7, 7),
    nn.Flatten(),
    nn.Linear(128, 10)
)

model(x).shape
model.to(device)

Sequential(
  (0): ConvBlock(
    (conv): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (batch): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (1): ConvBlock(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (batch): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (2): ConvBlock(
    (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (batch): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (3): ConvBlock(
    (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (batch): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (4): ConvBlock(
    (conv): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (batch): BatchNorm2d(128, eps=1e-05, momentum=0.1, af

In [ ]:
loss_fn = torch.nn.CrossEntropyLoss(label_smoothing=0.3)

In [ ]:
opt = torch.optim.Adam(model.parameters())

In [ ]:
for epoch in range(10):
    loss_list = []
    acc_list = []   
    model.train()
    for step, (x, y) in enumerate(dataloader_train):
        #1. feed forward (예측 결과를 계산한다.)
        pred = model(x.to(device))
        #2. loss
        loss = loss_fn(pred, y.to(device))
        #3. grad
        opt.zero_grad()
        loss.backward()
        #4. model update (opt)
        opt.step()

        #번외.. 5. accuracy
        acc = torch.sum(pred.to('cpu').detach().argmax(dim=1) == y) / len(y)
        loss_list.append(loss.item())
        acc_list.append(acc.item())
        total_loss = round(sum(loss_list) / len(loss_list), 4)
        total_acc = round(sum(acc_list) / len(acc_list), 4)

        print(f'\repoch={epoch} step={step} loss={total_loss} acc={total_acc}', end='')
    print()
    loss_list = []
    acc_list = []  
    model.eval()
    for step, (x, y) in enumerate(dataloader_valid):
        #1. feed forward (예측 결과를 계산한다.)
        with torch.no_grad():
            pred = model(x.to(device))
        #2. loss
        loss = loss_fn(pred, y.to(device))

        #번외.. 5. accuracy
        acc = torch.sum(pred.to('cpu').detach().argmax(dim=1) == y) / len(y)
        loss_list.append(loss.item())
        acc_list.append(acc.item())
        total_loss = round(sum(loss_list) / len(loss_list), 4)
        total_acc = round(sum(acc_list) / len(acc_list), 4)

        print(f'\r                                                step={step} loss={total_loss} acc={total_acc}', end='')
    print()

epoch=0 step=1874 loss=1.4178 acc=0.8503
                                                step=312 loss=1.3703 acc=0.883
epoch=1 step=1874 loss=1.3416 acc=0.9048
                                                step=312 loss=1.3338 acc=0.9052
epoch=2 step=1874 loss=1.3208 acc=0.9194
                                                step=312 loss=1.3346 acc=0.9012
epoch=3 step=1874 loss=1.3067 acc=0.9283
                                                step=312 loss=1.3235 acc=0.9146
epoch=4 step=1874 loss=1.2964 acc=0.9354
                                                step=312 loss=1.3099 acc=0.9212
epoch=5 step=1874 loss=1.2866 acc=0.9429
                                                step=312 loss=1.3133 acc=0.9185
epoch=6 step=1874 loss=1.2784 acc=0.9493
                                                step=312 loss=1.3015 acc=0.9242
epoch=7 step=1874 loss=1.2707 acc=0.9558
                                                step=312 loss=1.2992 acc=0.9269
epoch=8 step=1874 loss=1.2636 acc

In [ ]:
a = torch.randn(128, 1, 1)
a.reshape(-1).shape

torch.Size([128])

In [ ]:
model.eval()
with torch.no_grad():
    logit = model(x_valid.unsqueeze(1).to(device))
prob = torch.softmax(logit, dim=1)
y_pred = torch.argmax(prob, dim=1)
y_pred[:10]

tensor([9, 2, 1, 1, 6, 1, 4, 6, 5, 7], device='cuda:0')

In [ ]:
y_valid[:10]

In [ ]:
sum(y_pred == y_valid) / len(y_valid)